In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 10.6 MB/s eta 0:00:00


In [3]:
from sklearn.model_selection import train_test_split
from datasets import load_dataset, Dataset
import torch
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, GPT2Model
from sentence_transformers import SentenceTransformer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from bayes_opt import BayesianOptimization
import random
from sklearn.metrics.pairwise import cosine_similarity

2024-07-29 16:26:34.093369: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-29 16:26:34.093474: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-29 16:26:34.202327: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
# set the random seed
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    
    
# Load the dataset
dataset = load_dataset("SetFit/bbc-news")

# Convert the training and test datasets to DataFrames
train_df = dataset['train'].to_pandas()
test_df = dataset['test'].to_pandas()

# Combine the training and test DataFrames
all_df = pd.concat([train_df, test_df])

# 30 % data are used for test
train_df, test_df = train_test_split(all_df, test_size=0.2, random_state=seed)

#  20% of training set are divided to be validation set
train_df, val_df = train_test_split(train_df, test_size=0.125, random_state=seed)

# turn DataFrames into Dataset object
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)



Generating train split:   0%|          | 0/1225 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [5]:
print("Number of train data: ", len(train_dataset))
print("Number of val data: ", len(val_dataset))
print("Number of test data: ", len(test_dataset))

Number of train data:  1557
Number of val data:  223
Number of test data:  445


In [6]:


# Load GPT2Tokenizer and Model
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')



# Add pad_token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

model = GPT2ForSequenceClassification.from_pretrained('distilgpt2', num_labels=5)

# Resize token embeddings to match the tokenizer length
model.resize_token_embeddings(len(tokenizer))

# Set the padding token ID in the model configuration
model.config.pad_token_id = tokenizer.pad_token_id

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = model.to(device)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at distilgpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding=True, max_length=512)

train_data1 = train_dataset.map(tokenize_function, batched=True)
val_data1 = val_dataset.map(tokenize_function, batched=True)
test_data1 = test_dataset.map(tokenize_function, batched=True)

# Data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Define compute metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir='./logs',
    load_best_model_at_end=True,
    report_to="none",
    seed=seed,
)

# Define Trainer
trainer1 = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data1,
    eval_dataset=val_data1,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Train the model
trainer1.train()

# Evaluate the best model on the test set
eval_result = trainer1.evaluate(test_data1)
print(f"Final evaluation results on test set: {eval_result}")

Map:   0%|          | 0/1557 [00:00<?, ? examples/s]

Map:   0%|          | 0/223 [00:00<?, ? examples/s]

Map:   0%|          | 0/445 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.357079,0.932735,0.932773,0.941962,0.932735
2,No log,0.164561,0.973094,0.972913,0.973114,0.973094
3,No log,0.202698,0.973094,0.972906,0.973465,0.973094
4,No log,0.134134,0.973094,0.973002,0.973184,0.973094
5,No log,0.172868,0.973094,0.973006,0.973118,0.973094
6,0.274400,0.162212,0.973094,0.973002,0.973184,0.973094
7,0.274400,0.195228,0.959641,0.959616,0.960731,0.959641
8,0.274400,0.189160,0.973094,0.972998,0.973309,0.973094
9,0.274400,0.196428,0.973094,0.972998,0.973309,0.973094
10,0.274400,0.194779,0.973094,0.972998,0.973309,0.973094


Final evaluation results on test set: {'eval_loss': 0.15737980604171753, 'eval_accuracy': 0.9707865168539326, 'eval_f1': 0.9708435085090811, 'eval_precision': 0.9715983068651995, 'eval_recall': 0.9707865168539326, 'eval_runtime': 4.5225, 'eval_samples_per_second': 98.396, 'eval_steps_per_second': 6.191, 'epoch': 10.0}


In [8]:
enhanced_prompts = [
"What is the central focus of this newspaper article?",
"How would you formulate the main theme of this message?",
"Where is this story most likely to be found in the newspaper?",
"How would you define the primary subject of this news?",
"What is the main topic of this news?",
"Which category is best suited for this news?",
"What is the main topic of this news article?",
"What is central to this news article?",
"Which part of the newspaper is most likely to contain this news?",
"Where in the newspaper is this news most likely to be located?",
"How would you explain the main topic of this news?",
"Which section of the newspaper is this news most likely to be found in?",
"In what section of the newspaper is this news most likely to be found?",
"Which category is the best match for this news article?",
"In which category does this news article best fit?",
"Which category does this news fall into?",
"Which category is most appropriate for this news?",
"What category does this news belong to?",
"Which section of the newspaper will most likely contain this news?",
"In which section of the newspaper is this news story most likely to appear?",
"What is the central purpose of this news article?",
"In which section of the newspaper would one likely find this news item?",
"How would you summarize the central theme of this message?",
"In which section of the newspaper is this news most likely?",
"Which part of the newspaper is likely to contain this news?",
"Which category does this news best fit into?",
"How would you define the main topic of this news?",
"Which category best matches this news article?",
"What is the main emphasis of this news article?",
"How would you describe the main topic of this news?",
"In which section of the newspaper would this news item most likely be found?",
"How would you characterize the main theme of this message?",
"Where in the newspaper is this news story most likely to be found?",
"What is your description of the main theme of this news?",
"Which category is best for this news article?",
"What is the main point of this press article?",
"In your opinion, what is the main objective of this news?",
"In which section of the newspaper is this news most likely to appear?",
"What category does this news article fall under?",
"In what section of the newspaper is this news most likely to appear?",
"Under which category does this news article fit best?",
"What category does this news article belong to?",
"Where in the newspaper is this news most likely to appear?",
"What is your description of the main topic of this news story?",
"In which section of the newspaper would this news most likely be found?",
"How would you describe the main theme of this message?",
"Which section of the newspaper is most likely to contain this news?",
"Where in the newspaper is this news most likely to be found?",
"In which section of the newspaper is this news most likely to be found?",
"In which section of the newspaper are you most likely to find this news?",
"What is this news article focused on?",
"In what part of the newspaper would this news most likely appear?",
"How would you articulate the primary subject of this news?",
"What do you think is the main focus of this message?",
"What is the main emphasis of this news?",
"How would you define the main theme of this message?",
"Which part of the newspaper would most likely contain this news?",
"Which category best fits this news?",
"How would you explain the main theme of this news?",
"Which category is best suited for this news article?",
"What do you think is the main focus of this news?",
"Which category does this news article best fit into?",
"How would you describe the main topic of this news story?",
"What is the main focus of this news?",
"In which part of the newspaper is this news likely to be found?",
"In which category does this news article belong?",
"Which category best suits this news article?",
"What do you think is the main objective of this news?",
"How would you summarize the central theme of this news?",
"What would you say is the primary focus of this news?",
"How do you describe the main topic of this news story?",
"How would you characterize the main topic of this news story?",
"How would you describe the main theme of this news story?",
"What is the primary focus of this news article?",
"How would you define the primary topic of this news item?",
"Which section of the newspaper is most likely to publish this story?",
"What is the main focus of this news article?",
"What is the main theme of this news?",
"In which category does this news article fit best?",
"What is your description of the main subject of this news?",
"What is your description of the main topic of this news?",
"How would you articulate the main subject of this news?",
"How would you explain the main subject of this news?",
"What category does this newspaper article belong to?",
"Which section of the newspaper is most likely to contain this news story?",
"How would you phrase the main subject of this news story?",
"Where in the newspaper might this news be found?",
"What is the central point of this news article?",
"Which category best fits this news article?",
"What is the main purpose of this press article?",
"What is the main subject of this news article?",
"How would you characterize the main subject of this news?",
"Which section of the newspaper is most likely to feature this news?",
"In which part of the newspaper is this news most likely to be found?",
"In which section of the newspaper is it most likely to find this news?",
"In which part of the newspaper is this news most likely to appear?",
"What is the main topic of this article?",
"What category does this news article fall into?",
"Into which category does this news article best fit?",
"Which category is most suitable for this news article?",
"How would you phrase the primary topic of this news story?",
"In which section of the newspaper would this news likely be found?",
"What would you say is the main focus of this news?",
"How would you explain the main theme of this message?",
"In which part of the newspaper would this news most likely appear?",
"Which category does this news article fall under?",
"How would you describe the main theme of this news?",
"What is the main point of this news?",
"What is the key focus of this news article?",
"In which section of the newspaper would this news story likely be found?",
"How would you articulate the main point of this news story?",
"Which part of the newspaper would most likely contain this news story?",
"What is the central focus of this news article?",
"What is the main point of this news article?",
"What category is this news article in?",
"How would you characterize the main topic of this news?",
"Under which category does this news article best fit?",
"What category best fits this news article?",
"Which category is the best match for this news story?",
"Which section of the newspaper is this news most likely to be in?",
"How would you define the main subject of this news?",
"Which part of the newspaper would most likely feature this news?",
"In which section of the newspaper is this news story most likely to be found?",
"Which category does this news article best fit under?",
"How would you summarize the central theme of this news story?",
"What is the focus of this news article?",
"How would you articulate the main theme of this news story?",
"Which category is most appropriate for this news article?",
"How would you explain the main topic of this news story?",
"Which category does this news article belong to?",
"In which section of the newspaper is this news most likely to be published?",
"How would you describe the main subject of this news?",
"How would you express the main topic of this news?",
"What is the central focus of this news?",
"How would you articulate the main topic of this news?"
]

enhanced_prompts1= [ 
"What category best fits this news article?",
"How would you describe the main topic of this news?",
"In which section of the newspaper would this news likely be found?",
"What is the primary focus of this news article?",
"Which category does this news article fall under?",
"In which category does this news article belong?",
"Which category is most appropriate for this news article?",
"Under which category does this news article best fit?",
"Which category is the best match for this news article?",
"Into which category does this news article best fit?",
"Which category is most suitable for this news article?",
"How would you define the primary subject of this news?",
"How would you summarize the central theme of this news?",
"What would you say is the primary focus of this news?",
"How would you explain the main subject of this news?",
"How would you characterize the main topic of this news?",
"What is your description of the main topic of this news?",
"How would you articulate the primary subject of this news?",
"Which section of the newspaper is most likely to feature this news?",
"In what section of the newspaper is this news most likely to appear?",
"Which section of the newspaper is this news most likely to be found in?",
"Which part of the newspaper would most likely feature this news?",
"Which section of the newspaper is most likely to contain this news?",
"In what section of the newspaper is this news most likely to be found?",
"Where in the newspaper is this news most likely to be located?",
"What is the main emphasis of this news article?",
"What is the central focus of this news article?",
"What is the main point of this news article?",
"What is the main subject of this news article?",
"What is the main focus of this news article?",
"What is the key focus of this news article?",
"What is the main topic of this news article?",
"What category best fits this news article: business, entertainment, politics, sport or tech?",
"How would you describe the main topic of this news: business, entertainment, politics, sport or tech?",
"In which section of the newspaper would this news likely be found: business, entertainment, politics, sport or tech?",
"What is the primary focus of this news article: business, entertainment, politics, sport or tech?",
"Which category does this news article fall under: business, entertainment, politics, sport or tech?",
"In which category does this news article belong: business, entertainment, politics, sport or tech?",
"Which category is most appropriate for this news article: business, entertainment, politics, sport or tech?",
"Under which category does this news article best fit: business, entertainment, politics, sport or tech?",
"Which category is the best match for this news article: business, entertainment, politics, sport or tech?",
"Into which category does this news article best fit: business, entertainment, politics, sport or tech?",
"Which category is most suitable for this news article: business, entertainment, politics, sport or tech?",
"How would you define the primary subject of this news: business, entertainment, politics, sport or tech?",
"How would you summarize the central theme of this news: business, entertainment, politics, sport or tech?",
"What would you say is the primary focus of this news: business, entertainment, politics, sport or tech?",
"How would you explain the main subject of this news: business, entertainment, politics, sport or tech?",
"How would you characterize the main topic of this news: business, entertainment, politics, sport or tech?",
"What is your description of the main topic of this news: business, entertainment, politics, sport or tech?",
"How would you articulate the primary subject of this news: business, entertainment, politics, sport or tech?",
"Which section of the newspaper is most likely to feature this news: business, entertainment, politics, sport or tech?",
"In what section of the newspaper is this news most likely to appear: business, entertainment, politics, sport or tech?",
"Which section of the newspaper is this news most likely to be found in: business, entertainment, politics, sport or tech?",
"Which part of the newspaper would most likely feature this news: business, entertainment, politics, sport or tech?",
"Which section of the newspaper is most likely to contain this news: business, entertainment, politics, sport or tech?",
"In what section of the newspaper is this news most likely to be found: business, entertainment, politics, sport or tech?",
"Where in the newspaper is this news most likely to be located: business, entertainment, politics, sport or tech?",
"What is the main emphasis of this news article: business, entertainment, politics, sport or tech?",
"What is the central focus of this news article: business, entertainment, politics, sport or tech?",
"What is the main point of this news article: business, entertainment, politics, sport or tech?",
"What is the main subject of this news article: business, entertainment, politics, sport or tech?",
"What is the main focus of this news article: business, entertainment, politics, sport or tech?",
"What is the key focus of this news article: business, entertainment, politics, sport or tech?",
"What is the main topic of this news article: business, entertainment, politics, sport or tech?",
"What category best fits this news article: business, entertainment, politics, sport or tech? Example: 'wales want rugby league training wales could follow england s lead by training with a rugby league club' is sport.",
"How would you describe the main topic of this news: business, entertainment, politics, sport or tech? Example: 'markets signal brazilian recovery the brazilian stock market has risen to a record high as investors display growing confidence in the durability of the country s economic recovery' is business.",
"In which section of the newspaper would this news likely be found: business, entertainment, politics, sport or tech? Example: 'wales want rugby league training wales could follow england s lead by training with a rugby league club' is sport.",
"What is the primary focus of this news article: business, entertainment, politics, sport or tech? Example: 'markets signal brazilian recovery the brazilian stock market has risen to a record high as investors display growing confidence in the durability of the country s economic recovery' is business.",
"Which category does this news article fall under: business, entertainment, politics, sport or tech? Example: 'wales want rugby league training wales could follow england s lead by training with a rugby league club' is sport.",
"In which category does this news article belong: business, entertainment, politics, sport or tech? Example: 'wales want rugby league training wales could follow england s lead by training with a rugby league club' is sport.",
"Which category is most appropriate for this news article: business, entertainment, politics, sport or tech? Example: 'tough rules for ringtone sellers firms that flout rules on how ringtones and other mobile extras are sold could be cut off from all uk phone networks' is tech.",
"Under which category does this news article best fit: business, entertainment, politics, sport or tech? Example: 'wales want rugby league training wales could follow england s lead by training with a rugby league club' is sport.",
"Which category is the best match for this news article: business, entertainment, politics, sport or tech? Example: 'tough rules for ringtone sellers firms that flout rules on how ringtones and other mobile extras are sold could be cut off from all uk phone networks' is tech.",
"Into which category does this news article best fit: business, entertainment, politics, sport or tech? Example: 'rock band u2 break ticket record u2 have smashed irish box office records with ticket sales for their dublin concerts after more than 150 000 were sold within 50 minutes' is entertainment.",
"Which category is most suitable for this news article: business, entertainment, politics, sport or tech? Example: 'markets signal brazilian recovery the brazilian stock market has risen to a record high as investors display growing confidence in the durability of the country s economic recovery' is business.",
"How would you define the primary subject of this news: business, entertainment, politics, sport or tech? Example: 'iraq advice claim sparks new row the tories say ministers must respond in parliament to claims that the legal advice used to justify the iraq war was drawn up at number 10' is politics.",
"How would you summarize the central theme of this news: business, entertainment, politics, sport or tech? Example: 'rock band u2 break ticket record u2 have smashed irish box office records with ticket sales for their dublin concerts after more than 150 000 were sold within 50 minutes' is entertainment.",
"What would you say is the primary focus of this news: business, entertainment, politics, sport or tech? Example: 'wales want rugby league training wales could follow england s lead by training with a rugby league club' is sport.",
"How would you explain the main subject of this news: business, entertainment, politics, sport or tech? Example: 'iraq advice claim sparks new row the tories say ministers must respond in parliament to claims that the legal advice used to justify the iraq war was drawn up at number 10' is politics.",
"How would you characterize the main topic of this news: business, entertainment, politics, sport or tech? Example: 'tough rules for ringtone sellers firms that flout rules on how ringtones and other mobile extras are sold could be cut off from all uk phone networks' is tech.",
"What is your description of the main topic of this news: business, entertainment, politics, sport or tech? Example: 'rock band u2 break ticket record u2 have smashed irish box office records with ticket sales for their dublin concerts after more than 150 000 were sold within 50 minutes' is entertainment.",
"How would you articulate the primary subject of this news: business, entertainment, politics, sport or tech? Example: 'iraq advice claim sparks new row the tories say ministers must respond in parliament to claims that the legal advice used to justify the iraq war was drawn up at number 10' is politics.",
"Which section of the newspaper is most likely to feature this news: business, entertainment, politics, sport or tech? Example: 'tough rules for ringtone sellers firms that flout rules on how ringtones and other mobile extras are sold could be cut off from all uk phone networks' is tech.",
"In what section of the newspaper is this news most likely to appear: business, entertainment, politics, sport or tech? Example: 'wales want rugby league training wales could follow england s lead by training with a rugby league club' is sport.",
"Which section of the newspaper is this news most likely to be found in: business, entertainment, politics, sport or tech? Example: 'iraq advice claim sparks new row the tories say ministers must respond in parliament to claims that the legal advice used to justify the iraq war was drawn up at number 10' is politics.",
"Which part of the newspaper would most likely feature this news: business, entertainment, politics, sport or tech? Example: 'tough rules for ringtone sellers firms that flout rules on how ringtones and other mobile extras are sold could be cut off from all uk phone networks' is tech.",
"Which section of the newspaper is most likely to contain this news: business, entertainment, politics, sport or tech? Example: 'markets signal brazilian recovery the brazilian stock market has risen to a record high as investors display growing confidence in the durability of the country s economic recovery' is business.",
"In what section of the newspaper is this news most likely to be found: business, entertainment, politics, sport or tech? Example: 'iraq advice claim sparks new row the tories say ministers must respond in parliament to claims that the legal advice used to justify the iraq war was drawn up at number 10' is politics.",
"Where in the newspaper is this news most likely to be located: business, entertainment, politics, sport or tech? Example: 'wales want rugby league training wales could follow england s lead by training with a rugby league club' is sport.",
"What is the main emphasis of this news article: business, entertainment, politics, sport or tech? Example: 'wales want rugby league training wales could follow england s lead by training with a rugby league club' is sport.",
"What is the central focus of this news article: business, entertainment, politics, sport or tech? Example: 'rock band u2 break ticket record u2 have smashed irish box office records with ticket sales for their dublin concerts after more than 150 000 were sold within 50 minutes' is entertainment.",
"What is the main point of this news article: business, entertainment, politics, sport or tech? Example: 'wales want rugby league training wales could follow england s lead by training with a rugby league club' is sport.",
"What is the main subject of this news article: business, entertainment, politics, sport or tech? Example: 'iraq advice claim sparks new row the tories say ministers must respond in parliament to claims that the legal advice used to justify the iraq war was drawn up at number 10' is politics.",
"What is the main focus of this news article: business, entertainment, politics, sport or tech? Example: 'wales want rugby league training wales could follow england s lead by training with a rugby league club' is sport.",
"What is the key focus of this news article: business, entertainment, politics, sport or tech? Example: 'iraq advice claim sparks new row the tories say ministers must respond in parliament to claims that the legal advice used to justify the iraq war was drawn up at number 10' is politics.",
"What is the main topic of this news article: business, entertainment, politics, sport or tech? Example: 'wales want rugby league training wales could follow england s lead by training with a rugby league club' is sport." ]

In [9]:
# load Sentence Transformer model
embedmodel = SentenceTransformer('all-MiniLM-L6-v2')

# generate embeddings for each prompt
prompt_embeddings = embedmodel.encode(enhanced_prompts)
prompt_embeddings1 = embedmodel.encode(enhanced_prompts1)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/5 [00:00<?, ?it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
# classify function
def classify_article_with_prompt_embedding(prompt_embedding, article_text):
    prompt_text = " ".join(map(str, prompt_embedding))
    input_text = f"{prompt_text} {article_text}"
    inputs = tokenizer(input_text, truncation=True, max_length=512, padding='max_length', return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    label_id = outputs.logits.argmax(dim=1).item()
    return label_id

def evaluate_prompt_embedding(prompt_embedding):
    predictions = [classify_article_with_prompt_embedding(prompt_embedding, article_text) for article_text in train_dataset['text']]
    return accuracy_score(train_dataset['label'], predictions)

# Bayesian optimization function
def black_box_function(prompt_idx):
    prompt_embedding = prompt_embeddings[int(prompt_idx)]
    return evaluate_prompt_embedding(prompt_embedding)

pbounds = {'prompt_idx': (0, len(prompt_embeddings) - 1)}

optimizer = BayesianOptimization(
    f=black_box_function,
    pbounds=pbounds,
    random_state=42,
)

optimizer.maximize(
    init_points=5,
    n_iter=15,
)

|   iter    |  target   | prompt... |
-------------------------------------
| 1         | 0.1843    | 50.19     |
| 2         | 0.1747    | 127.4     |
| 3         | 0.2306    | 98.09     |
| 4         | 0.1747    | 80.22     |
| 5         | 0.1747    | 20.91     |
| 6         | 0.2306    | 98.04     |
| 7         | 0.1747    | 109.6     |
| 8         | 0.2351    | 0.0       |
| 9         | 0.2306    | 6.37      |
| 10        | 0.1843    | 64.38     |
| 11        | 0.1747    | 36.59     |
| 12        | 0.1843    | 2.929     |
| 13        | 0.1747    | 8.105     |
| 14        | 0.1843    | 5.393     |
| 15        | 0.2351    | 0.6095    |
| 16        | 0.2351    | 99.39     |
| 17        | 0.1747    | 100.5     |
| 18        | 0.1843    | 96.86     |
| 19        | 0.2306    | 98.88     |
| 20        | 0.2306    | 1.432     |


In [11]:
# Step 1: Find the maximum target value
max_target_value = max(res['target'] for res in optimizer.res)

# Step 2: Collect all unique prompt indices with the maximum target value
best_prompt_indices = [int(res['params']['prompt_idx']) for res in optimizer.res if res['target'] == max_target_value]
unique_best_prompt_indices = list(set(best_prompt_indices))

# Step 3: Get the unique embeddings
best_prompt_embeddings = [prompt_embeddings[idx] for idx in unique_best_prompt_indices]

# Step 4: Find the top one most similar prompt for each of the best embeddings
all_top_similar_prompts_dict = {}

for idx, best_prompt_embedding in enumerate(best_prompt_embeddings):
    similarities = cosine_similarity(best_prompt_embedding[np.newaxis, :], prompt_embeddings)[0]
    # Get the index of the most similar prompt for each best embedding
    top_index = np.argmax(similarities)
    # Collect the most similar prompt
    top_similar_prompt = enhanced_prompts[top_index]
    all_top_similar_prompts_dict[unique_best_prompt_indices[idx]] = top_similar_prompt

# Output best prompt embedding indices and their corresponding top one similar prompts
print("Best prompt embedding indices and their top one similar prompts:")
for index, prompt in all_top_similar_prompts_dict.items():
    print(f"Index: {index}, Top Similar Prompt: {prompt}")

Best prompt embedding indices and their top one similar prompts:
Index: 0, Top Similar Prompt: What is the central focus of this newspaper article?
Index: 99, Top Similar Prompt: Which category is most suitable for this news article?


In [12]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir='./logs',
    load_best_model_at_end=True,
    report_to="none",
    seed= seed,
)

# Data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Iterate over each index and its most similar prompt
for idx, prompts in all_top_similar_prompts_dict.items():
    best_prompt = prompts  # Since we now store only the top prompt per index
    print(f"Training and evaluating for best prompt embedding index: {idx}")
    print(f"Most similar prompt: {best_prompt}")

    # Function to tokenize text with the given prompt
    def tokenize_with_prompt(examples):
        inputs = [f"{best_prompt} {text}" for text in examples['text']]
        return tokenizer(inputs, truncation=True, padding='max_length', max_length=512)

    # Apply the tokenize function to datasets
    train_data_with_prompt = train_dataset.map(tokenize_with_prompt, batched=True)
    val_data_with_prompt = val_dataset.map(tokenize_with_prompt, batched=True)
    test_data_with_prompt = test_dataset.map(tokenize_with_prompt, batched=True)

    # Initialize the Trainer
    trainer2 = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_data_with_prompt,
        eval_dataset= val_data_with_prompt,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics
    )

    # Train the model
    trainer2.train()

    # Evaluate the model on the test set
    eval_result = trainer2.evaluate(test_data_with_prompt)
    print(f"Final evaluation results for prompt index {idx} on test set: {eval_result}")

Training and evaluating for best prompt embedding index: 0
Most similar prompt: What is the central focus of this newspaper article?


/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map:   0%|          | 0/1557 [00:00<?, ? examples/s]

Map:   0%|          | 0/223 [00:00<?, ? examples/s]

Map:   0%|          | 0/445 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/1557 [00:00<?, ? examples/s]

Map:   0%|          | 0/223 [00:00<?, ? examples/s]

Map:   0%|          | 0/445 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.354597,0.941704,0.942078,0.943757,0.941704
2,No log,0.291746,0.973094,0.972914,0.973391,0.973094
3,No log,0.298831,0.959641,0.959543,0.960065,0.959641
4,No log,0.263637,0.964126,0.963894,0.963857,0.964126
5,No log,0.447882,0.937220,0.937672,0.942715,0.937220
6,0.013800,0.291979,0.964126,0.963945,0.964037,0.964126
7,0.013800,0.373214,0.950673,0.950518,0.951479,0.950673
8,0.013800,0.368375,0.950673,0.950518,0.951479,0.950673
9,0.013800,0.352266,0.959641,0.959481,0.959633,0.959641
10,0.013800,0.348289,0.959641,0.959481,0.959633,0.959641


Final evaluation results for prompt index 99 on test set: {'eval_loss': 0.1525474637746811, 'eval_accuracy': 0.9865168539325843, 'eval_f1': 0.9865610120512225, 'eval_precision': 0.987067728343116, 'eval_recall': 0.9865168539325843, 'eval_runtime': 4.5692, 'eval_samples_per_second': 97.391, 'eval_steps_per_second': 6.128, 'epoch': 10.0}


In [13]:
def classify_article_with_prompt_embedding(prompt_embedding, article_text):
    prompt_text = " ".join(map(str, prompt_embedding))
    input_text = f"{prompt_text} {article_text}"
    inputs = tokenizer(input_text, truncation=True, max_length=512, padding='max_length', return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    label_id = outputs.logits.argmax(dim=1).item()
    return label_id

def evaluate_prompt_embedding(prompt_embedding):
    predictions = [classify_article_with_prompt_embedding(prompt_embedding, article_text) for article_text in train_dataset['text']]
    return accuracy_score(train_dataset['label'], predictions)

# Bayesian optimization function
def black_box_function(prompt_idx):
    prompt_embedding = prompt_embeddings1[int(prompt_idx)]
    return evaluate_prompt_embedding(prompt_embedding)

pbounds = {'prompt_idx': (0, len(prompt_embeddings1) - 1)}

optimizer = BayesianOptimization(
    f=black_box_function,
    pbounds=pbounds,
    random_state=42,
)

optimizer.maximize(
    init_points=5,
    n_iter=15,
)

|   iter    |  target   | prompt... |
-------------------------------------
| 1         | 0.1747    | 35.58     |
| 2         | 0.1753    | 90.32     |
| 3         | 0.1747    | 69.54     |
| 4         | 0.1747    | 56.87     |
| 5         | 0.2351    | 14.82     |
| 6         | 0.2306    | 5.559     |
| 7         | 0.1843    | 13.78     |
| 8         | 0.1753    | 34.66     |
| 9         | 0.1747    | 15.18     |
| 10        | 0.2351    | 89.57     |
| 11        | 0.1747    | 88.27     |
| 12        | 0.2306    | 31.02     |
| 13        | 0.2306    | 5.559     |
| 14        | 0.2351    | 14.65     |
| 15        | 0.2351    | 89.29     |
| 16        | 0.2351    | 30.63     |
| 17        | 0.2351    | 30.18     |
| 18        | 0.1843    | 29.68     |
| 19        | 0.2306    | 5.079     |
| 20        | 0.2306    | 4.559     |


In [14]:
# Step 1: Find the maximum target value
max_target_value = max(res['target'] for res in optimizer.res)

# Step 2: Collect all unique prompt indices with the maximum target value
best_prompt_indices = [int(res['params']['prompt_idx']) for res in optimizer.res if res['target'] == max_target_value]
unique_best_prompt_indices = list(set(best_prompt_indices))

# Step 3: Get the unique embeddings
best_prompt_embeddings = [prompt_embeddings1[idx] for idx in unique_best_prompt_indices]

# Step 4: Find the top one most similar prompt for each of the best embeddings
all_top_similar_prompts_dict = {}

for idx, best_prompt_embedding in enumerate(best_prompt_embeddings):
    similarities = cosine_similarity(best_prompt_embedding[np.newaxis, :], prompt_embeddings1)[0]
    # Get the index of the most similar prompt for each best embedding
    top_index = np.argmax(similarities)
    # Collect the most similar prompt
    top_similar_prompt = enhanced_prompts1[top_index]
    all_top_similar_prompts_dict[unique_best_prompt_indices[idx]] = top_similar_prompt

# Output best prompt embedding indices and their corresponding top one similar prompts
print("Best prompt embedding indices and their top one similar prompts:")
for index, prompt in all_top_similar_prompts_dict.items():
    print(f"Index: {index}, Top Similar Prompt: {prompt}")

Best prompt embedding indices and their top one similar prompts:
Index: 89, Top Similar Prompt: What is the main emphasis of this news article: business, entertainment, politics, sport or tech? Example: 'wales want rugby league training wales could follow england s lead by training with a rugby league club' is sport.
Index: 14, Top Similar Prompt: How would you explain the main subject of this news?
Index: 30, Top Similar Prompt: What is the key focus of this news article?


In [15]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir='./logs',
    load_best_model_at_end=True,
    report_to="none",
    seed= seed,
)
# Data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Iterate over each index and its most similar prompt
for idx, prompts in all_top_similar_prompts_dict.items():
    best_prompt = prompts  # Since we now store only the top prompt per index
    print(f"Training and evaluating for best prompt embedding index: {idx}")
    print(f"Most similar prompt: {best_prompt}")

    # Function to tokenize text with the given prompt
    def tokenize_with_prompt(examples):
        inputs = [f"{best_prompt} {text}" for text in examples['text']]
        return tokenizer(inputs, truncation=True, padding='max_length', max_length=512)

    # Apply the tokenize function to datasets
    train_data_with_prompt1 = train_dataset.map(tokenize_with_prompt, batched=True)
    val_data_with_prompt1 = val_dataset.map(tokenize_with_prompt, batched=True)
    test_data_with_prompt1 = test_dataset.map(tokenize_with_prompt, batched=True)

    # Initialize the Trainer
    trainer3 = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_data_with_prompt1,
        eval_dataset=val_data_with_prompt1,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics
    )

    # Train the model
    trainer3.train()

    # Evaluate the model on the test set
    eval_result = trainer3.evaluate(test_data_with_prompt1)
    print(f"Final evaluation results for prompt index {idx} on test set: {eval_result}")

Training and evaluating for best prompt embedding index: 89
Most similar prompt: What is the main emphasis of this news article: business, entertainment, politics, sport or tech? Example: 'wales want rugby league training wales could follow england s lead by training with a rugby league club' is sport.


/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map:   0%|          | 0/1557 [00:00<?, ? examples/s]

Map:   0%|          | 0/223 [00:00<?, ? examples/s]

Map:   0%|          | 0/445 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/1557 [00:00<?, ? examples/s]

Map:   0%|          | 0/223 [00:00<?, ? examples/s]

Map:   0%|          | 0/445 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.290344,0.977578,0.977245,0.978625,0.977578
2,No log,0.306624,0.968610,0.968517,0.968513,0.968610
3,No log,0.335015,0.968610,0.968380,0.968806,0.968610
4,No log,0.315152,0.973094,0.973006,0.973118,0.973094
5,No log,0.310483,0.977578,0.977540,0.977704,0.977578
6,0.002600,0.270860,0.982063,0.981833,0.982660,0.982063
7,0.002600,0.282673,0.973094,0.973006,0.973118,0.973094
8,0.002600,0.279937,0.973094,0.973006,0.973118,0.973094
9,0.002600,0.298925,0.973094,0.973006,0.973118,0.973094
10,0.002600,0.299175,0.973094,0.973006,0.973118,0.973094


Final evaluation results for prompt index 30 on test set: {'eval_loss': 0.175344318151474, 'eval_accuracy': 0.9775280898876404, 'eval_f1': 0.9775556584251022, 'eval_precision': 0.9777769666320002, 'eval_recall': 0.9775280898876404, 'eval_runtime': 4.5507, 'eval_samples_per_second': 97.787, 'eval_steps_per_second': 6.153, 'epoch': 10.0}
